In [1]:
import pandas as pd
import numpy as np

In [12]:
# Load the data
df = pd.read_excel('DVS-HP.xlsx')

# Filter for Combine Map
df = df[df['Map'] == 'Combine']

# Parse Score into numeric columns
df[['Score1', 'Score2']] = df['Score'].str.split('-', expand=True).astype(int)

# Compute points per hill for each team
df['Points1'] = df.groupby(['Map','Team1','Team2'])['Score1'].diff().fillna(df['Score1'])
df['Points2'] = df.groupby(['Map','Team1','Team2'])['Score2'].diff().fillna(df['Score2'])

# Prepare long format for points
long_pts = pd.concat([
    df[['Map','Hill','Team1','Points1']].rename(columns={'Team1':'Team','Points1':'Points'}),
    df[['Map','Hill','Team2','Points2']].rename(columns={'Team2':'Team','Points2':'Points'})
])

# Filter only for DVS and XLR8 teams
long_pts = long_pts[(long_pts['Team'] == 'DVS') | (long_pts['Team'] == 'XLR8')]
long_pts.head()

,Map,Hill,Team,Points
17,Combine,P1,DVS,21.0
18,Combine,P2,DVS,33.0
19,Combine,P3,DVS,33.0
20,Combine,P4,DVS,24.0
21,Combine,P1,DVS,12.0


In [24]:
# List of all teams
teams = sorted(long_pts['Team'].unique())

# Team-level stats
team_stats = []

for team in teams:
    played = df[(df['Team1']==team) | (df['Team2']==team)]
    # Rotation-win %
    rf = played[played['RotateFirst']==team]
    rot_rate = (rf['RotationWin']=='Yes').mean() * 100 if not rf.empty else np.nan
    # Break-success %
    opp = played[played['RotateFirst']!=team]
    breaks = played[(played['BreakTeam']==team) & (played['BreakSuccess']=='Yes')]
    break_rate = len(breaks) / len(opp) * 100 if len(opp)>0 else np.nan
    # Avg durations
    avg_hold = rf['HoldDuration'].mean()
    avg_break_hold = played[played['BreakTeam']==team]['BreakDuration'].mean()
    # Scrap points
    scrap_pts = df[df['ScrapTeam']==team]['ScrapTime'].sum()
    # Most dominant hill
    team_hill_avg = long_pts[long_pts['Team']==team].groupby('Hill')['Points'].mean()
    dominant_hill = team_hill_avg.idxmax()
    dominant_avg_pts = team_hill_avg.max()
    # Weakest hill
    weakest_hill = team_hill_avg.idxmin()
    weakest_avg_pts = team_hill_avg.min()
    team_stats.append({
        'Team': team,
        'RotationWin': rot_rate,
        'BreakSuccess': break_rate,
        'AvgHoldDuration (s)': avg_hold,
        'AvgBreakDuration (s)': avg_break_hold,
        'ScrapPoints': scrap_pts,
        'DominantHill': dominant_hill,
        'DominantAvgPoints': dominant_avg_pts,
        'WeakestHill': weakest_hill,
        'WeakestAvgPoints': weakest_avg_pts
    })

team_stats_df = pd.DataFrame(team_stats).set_index('Team')

In [25]:
team_stats_df.to_csv('team_stats.csv')

In [23]:
# Get average points per hill for each team
hill_avg = long_pts.groupby(['Hill', 'Team'])['Points'].mean().reset_index()
hill_avg = hill_avg.rename(columns={'Points': 'AvgPoints'})
hill_avg = hill_avg.pivot(index='Hill', columns='Team', values='AvgPoints').fillna(0)
hill_avg = hill_avg.reset_index()
hill_avg = hill_avg.rename_axis(None, axis=1)
hill_avg = hill_avg.sort_values(by='Hill')
hill_avg = hill_avg.round(2)
hill_avg.to_csv('hill_avg_points.csv', index=False)